In [ ]:
import numpy as np

def mdpgen(S, A):
    """
    Generate a random MDP with S states and A actions.

    Returns:
        P    (S, S, A) array  : transition kernels P(s' | s, a)
        R    (S, A)     array : rewards R(s, a) in [0, 10)
        Ppi  (S, S)     array : transition matrix under uniform policy
        p    (S,)       array : stationary distribution of Ppi
    """
    # 1) Random transition kernels, then normalize rows
    P = np.random.rand(S, S, A)
    for j in range(A):
        for i in range(S):
            P[i, :, j] /= P[i, :, j].sum()

    # 2) Uniform behavior policy transition matrix
    Ppi = P.sum(axis=2) / A

    # 3) Stationary distribution: left eigenvector of Ppi for eigenvalue 1
    #    which is the right eigenvector of Ppi.T
    eigvals, eigvecs = np.linalg.eig(Ppi.T)
    idx = np.argmin(np.abs(eigvals - 1.0))
    p = np.real(eigvecs[:, idx])
    # enforce non-negativity up to a sign
    p *= np.sign(p)
    p /= p.sum()

    # 4) Random reward function
    R = 10*np.random.rand(S, A)

    return P, R, Ppi, p

In [ ]:
S, A, gamma, T = 25, 4, 0.5, 50000
P, R, Ppi, p = mdpgen(S, A)
def value_iteration(P, R, gamma, T):
    """
    Run value iteration for T steps to compute optimal Q-function.
    Returns:
        Q     (S, A)   array : estimated state-action values
        Err   (T,)     array : infinity-norm Bellman residual at each iteration
    """
    S, _, A = P.shape
    Q = np.zeros((S, A))
    Err = np.zeros(T)
    for t in range(T):
        Q_old = Q.copy()
        V = Q_old.max(axis=1)  # state-value function
        # Bellman optimality update vectorized
        Q = R + gamma * np.tensordot(P, V, axes=([1], [0]))
        Err[t] = np.max(np.abs(Q - Q_old))
    return Q, Err

In [ ]:
# Regenerate everything and save to current working directory
# Generate MDP and optimal Q-values
#P, R = mdpgen(S, A)
Qopt,_ = value_iteration(P, R, gamma, T=250000)
#Qopt = np.array(Qopt)

In [ ]:
for name, val in [('P', P), ('R', R), ('p', p), ('Qopt', Qopt), ('gamma', gamma), ('S', S), ('A', A)]:
    print(f"{name}: type={type(val)}, shape={getattr(val, 'shape', 'N/A')}")


In [ ]:
np.savez('Qlearn_data_new.npz',
         P=P,
         R=R,
         p=p,
         Qopt=Qopt,
         gamma=gamma,
         S=np.int32(S),
         A=np.int32(A))